# Airbnb Notebook

# 1 - Limpeza de Dados

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/listings-full.csv', sep=',')
data.shape

/home/ramon/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (61,62,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(33715, 106)

Algumas colunas possuem preços no formato. x,xxx.xx. 
Vamos remover "$" e ","

In [2]:
cols = ['price', 'security_deposit', 'extra_people']
data[cols].head()

,price,security_deposit,extra_people
0,$332.00,$0.00,$63.00
1,$160.00,"$1,000.00",$45.00
2,$273.00,$0.00,$42.00
3,$378.00,"$1,050.00",$63.00
4,$130.00,$400.00,$80.00


In [3]:
cols = ['price', 'security_deposit', 'extra_people']
data[cols] = data[cols].apply(lambda x: x.str.replace('$','').str.replace(',','').astype(float))
data[cols].dtypes

price               float64
security_deposit    float64
extra_people        float64
dtype: object

Como pode ser visto abaixo, há colunas cujos valores não são númericos. Vamos manter apenas as colunas com valores numéricos e um subconjunto selecionado de colunas. Esse subconjunto contém colunas que acreditamos ser relevantes para o problema após consultar um especialista. Desse modo, mantivemos 63 das 106 colunas originais.

In [4]:
for k,v in zip(data.dtypes.index, data.dtypes):
    print('%s \t %s' %(k,v))

id 	 int64
listing_url 	 object
scrape_id 	 int64
last_scraped 	 object
name 	 object
summary 	 object
space 	 object
description 	 object
experiences_offered 	 object
neighborhood_overview 	 object
notes 	 object
transit 	 object
access 	 object
interaction 	 object
house_rules 	 object
thumbnail_url 	 float64
medium_url 	 float64
picture_url 	 object
xl_picture_url 	 float64
host_id 	 int64
host_url 	 object
host_name 	 object
host_since 	 object
host_location 	 object
host_about 	 object
host_response_time 	 object
host_response_rate 	 object
host_acceptance_rate 	 float64
host_is_superhost 	 object
host_thumbnail_url 	 object
host_picture_url 	 object
host_neighbourhood 	 object
host_listings_count 	 float64
host_total_listings_count 	 float64
host_verifications 	 object
host_has_profile_pic 	 object
host_identity_verified 	 object
street 	 object
neighbourhood 	 object
neighbourhood_cleansed 	 object
neighbourhood_group_cleansed 	 float64
city 	 object
state 	 object
zipcode 	 obj

In [5]:
df = data.select_dtypes(include=np.number)
selected = ['host_since', 'host_response_time', 'host_acceptance_rate', 'host_is_superhost', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'neighbourhood', 'neighbourhood_cleansed', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'weekly_price', 'monthly_price', 'security_deposit', 'security_deposit', 'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'number_of_reviews_ltm', 'first_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification', 'reviews_per_month', 'price', 'room_type']

cols = set(list(df.columns) + selected)
df = data[list(cols)]
df.shape

(33715, 63)

Como podemos ver abaixo, a coluna 'amenities', comodidades em tradução livre, possui uma lista de strings. Acreditamos que comodidades podem afetar diretamente o preço de aluguel.

In [6]:
df['amenities'].head()

0    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
1    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
2    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
3    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
4    {TV,"Cable TV",Wifi,"Air conditioning",Kitchen...
Name: amenities, dtype: object

Vamos criar uma nova coluna 'num_amenities' para representar o número de comodidades. Desse modo, partimos da hipótese de que um apartamento que forneça mais comodidades provavelmente terá um preço maior.

In [7]:
df['num_amenities'] = df['amenities'].apply(lambda x: len(x.split(',')))
df['num_amenities'].head()

/home/ramon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    33
1    14
2    25
3    33
4    41
Name: num_amenities, dtype: int64

Vamos agora analisar missing data. Primeiro, vamos avaliar a razão de missing data para cada coluna.

In [8]:
df.isnull().sum()/len(df)

neighbourhood_group_cleansed                    1.000000
jurisdiction_names                              1.000000
cancellation_policy                             0.000000
host_acceptance_rate                            1.000000
guests_included                                 0.000000
host_is_superhost                               0.000593
price                                           0.000000
medium_url                                      1.000000
longitude                                       0.000000
xl_picture_url                                  1.000000
latitude                                        0.000000
availability_30                                 0.000000
calculated_host_listings_count_shared_rooms     0.000000
id                                              0.000000
number_of_reviews                               0.000000
monthly_price                                   0.916209
maximum_minimum_nights                          0.000000
room_type                      

Podemos ver que há colunas em que todas as linhas são missing data. Portanto, vamos manter apenas colunas que possuem mais de 50% dos dados.

In [10]:
df = df[df.columns[df.count(axis=0)/len(df) > 0.5]]
df.isnull().sum()/len(df)

cancellation_policy                             0.000000
guests_included                                 0.000000
host_is_superhost                               0.000593
price                                           0.000000
longitude                                       0.000000
latitude                                        0.000000
availability_30                                 0.000000
calculated_host_listings_count_shared_rooms     0.000000
id                                              0.000000
number_of_reviews                               0.000000
maximum_minimum_nights                          0.000000
room_type                                       0.000000
host_since                                      0.000593
review_scores_rating                            0.457423
availability_90                                 0.000000
review_scores_checkin                           0.457986
bathrooms                                       0.001602
calculated_host_listings_count_

Em seguida, vamos remover algumas colunas, transformar valores booleanos em numéricos, remover dados com missing data no campo 'neighbourhood'. Por fim, vamos realizar imputação de dados. Na literatura de Estatística, há várias técnicas de imputação de dados (média, regressão, múltipla). Neste momento, vamos simplemente imputar missing data com valor zero. Posteriormente, pretendemos avaliar o efeito de imputar com a média.

In [13]:
to_drop = ['id', 'security_deposit', 'minimum_nights_avg_ntm', 'maximum_maximum_nights', 
           'neighbourhood_cleansed', 'host_id', 'scrape_id', 'host_since', 'host_response_time', 
           'maximum_minimum_nights', 'latitude',  'minimum_maximum_nights', 'minimum_minimum_nights', 
           'cancellation_policy', 'host_neighbourhood', 'maximum_nights_avg_ntm', 'amenities', 'longitude']

dff = df.drop(columns=to_drop)
dff = dff.replace(['f','t'],[0,1])
dff = dff[~dff['neighbourhood'].isna()]
dff.fillna(0, inplace=True)

dff.count(axis=0)/len(dff)

guests_included                                 1.0
host_is_superhost                               1.0
price                                           1.0
availability_30                                 1.0
calculated_host_listings_count_shared_rooms     1.0
number_of_reviews                               1.0
room_type                                       1.0
review_scores_rating                            1.0
availability_90                                 1.0
review_scores_checkin                           1.0
bathrooms                                       1.0
calculated_host_listings_count_private_rooms    1.0
calculated_host_listings_count_entire_homes     1.0
review_scores_location                          1.0
first_review                                    1.0
host_total_listings_count                       1.0
review_scores_value                             1.0
reviews_per_month                               1.0
review_scores_cleanliness                       1.0
host_listing

Agora, temos um dataset onde não há missing data e contém 37 das 106 colunas e 31757 das 33715 linhas originais. Este dataset reduzido será utilizado na etapa seguinte.

In [14]:
dff.shape

(31757, 37)

# 2- Análise Exploratória

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 28 06:28:50 2020

@author: ramon
"""
import pandas as pd
import numpy as np
np.random.seed(0)

import math

from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

from sklearn.neighbors import KNeighborsRegressor

from xgboost import XGBRegressor
from xgboost import plot_importance
 
from sklearn.ensemble import StackingRegressor

In [ ]:
## load data
name = 'data.csv'
y_name = 'price'

data = pd.read_csv(name, sep=',', header=0)

data.head(n=5)

In [ ]:
cnt_bed_type = data['bed_type'].value_counts()
(cnt_bed_type/np.sum(cnt_bed_type)).plot(kind='bar')

In [ ]:
data.shape

In [ ]:
data.dtypes